In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor

from parsers import vulnerability_parser, attack_graph_parser, topology_parser
from compose_generator import generate_full_conn
from mio import wrappers

__version__ = "1.0-dev2"

argv=['', 'examples/full-conn/10-example']

generate example folders of full connections

In [ ]:
x = list(range(50, 1001, 50))
x.append([1, 5, 10])
for j in x:
    generate_full_conn(j)

Initial Configurations

In [ ]:
stat, config, example_folder, result_folder, times = wrappers.init(argv)

assert stat == 0

Parsing the topology of docker-compose file

In [ ]:
networks, services, gateway_nodes, dt = topology_parser.parse_topology(example_folder)

networks: dict[str, dict[str, set]]
services: dict[str, dict[str, ]]
gateway_nodes: set[str]


create topology in format of dictionary of lists

In [ ]:
topology_graph, gateway_graph, gateway_graph_labels, dg = topology_parser.create_graphs(networks, services)

topology_graph: nx.Graph
gateway_graph: nx.Graph
gateway_graph_labels: dict[(str, str), str]

print('Time for generating topology:', dt + dg, 'seconds.')

In [ ]:
plt.axis("off")

plt.figure()
pos = nx.spring_layout(gateway_graph)
nx.draw_networkx_nodes(gateway_graph, pos)
nx.draw_networkx_edges(gateway_graph, pos)
nx.draw_networkx_labels(gateway_graph, pos)
nx.draw_networkx_edge_labels(gateway_graph, pos, edge_labels=gateway_graph_labels)
plt.show()

plt.figure()
pos = nx.spring_layout(topology_graph)
nx.draw_networkx_nodes(topology_graph, pos)
nx.draw_networkx_edges(topology_graph, pos)
nx.draw_networkx_labels(topology_graph, pos)
plt.show()


Parsing the vulnerabilities for each docker container

In [ ]:
stat, vulnerabilities, parsed_images, dv = vulnerability_parser. \
        parse_vulnerabilities(example_folder, services)

assert stat == 0

vulnerabilities: dict[str, dict[str, ]]
parsed_images: set[str]


Merging the attack vector files and creating an attack vector dictionary.

In [ ]:
exploitable_vulnerabilities, dvp = vulnerability_parser.get_exploitable_vulnerabilities(
        services, vulnerabilities, config["attack-vector-folder-path"], config["preconditions-rules"], config["postconditions-rules"])

exploitable_vulnerabilities: dict[str, dict[str, dict[str, int]]]

print('Time for vulnerability parser module:', dv + dvp, 'seconds.')

Getting the attack graph nodes and edges from the attack paths.

In [ ]:
executor = ProcessPoolExecutor()

attack_graph, da = attack_graph_parser.\
        generate_attack_graph(networks, services, exploitable_vulnerabilities, executor)

composed_graph, dcg = attack_graph_parser.get_graph_compose(attack_graph)

print("Time for attack graph generating module:", da + dcg, " seconds.")

attack_graph: dict[str, nx.DiGraph]

In [ ]:
wrappers.print_summary(topology_graph.number_of_nodes(),
                               topology_graph.number_of_edges(),
                               composed_graph.number_of_nodes(),
                               composed_graph.number_of_edges())


add container to a network

name = 'test'
image = 'nginx'
network = ['db', 'backend']

wrappers.add_node(name, image, network, vulnerabilities, parsed_images, example_folder, networks, topology,
         topology_graph, services, mapping_names, attack_vector_dict, config, nodes, edges,
         passed_nodes, passed_edges, attack_graph)


delete container by name

name = 'test'
wrappers.del_node(name, networks, topology, services, mapping_names, nodes, edges, passed_nodes, passed_edges, attack_graph)


add honeypot nodes


to = 'goal'
minimum = 3

path_counts = wrappers.gen_defence_list(attack_graph, to)
wrappers.deploy_honeypot(path_counts, minimum, vulnerabilities, parsed_images, example_folder, networks,
                         topology, topology_graph, services, mapping_names, attack_vector_dict, config,
                         nodes, edges, passed_nodes, passed_edges, attack_graph)